### Стратифицированные выборки

Предположим, что необходимо оценить число свободных мест в больницах. Все больницы разбили на 2 страты по числу коек-мест. Доля страты 1 составила 40%, страты 2 - 60%.

Из каждой страты были получены выборки числа свободных мест объемом 10 каждая:

<li> страта 1: 282, 226, 188, 327, 344, 304, 414, 224, 335, 270
<li> страта 2: 417,  851,  742, 1217, 1160,  993,  864,  852, 1286,  988
<br>
<p>Найдите оценку среднего числа свободных мест в больницах, а также оценку дисперсии числа свободных мест



<h4>Оценка среднего числа свободных мест</h4>
<p> $\large E\overline{X}=\sum ^{ℓ}_{k=1}W_{k}EX_{k}$
<br>
<h4>Оценка дисперсии числа свободных мест</h4>
<p> $\large D^{\ast }=\sum ^{l}_{k=1}W_{k}D_{k}^{\ast }+\sum ^{l}_{k=1}W_{k}\left( \sum ^{l}_{i=1}W_{i}X_{i}-\overline{X}_{k}\right) ^{2}$
    

In [1]:
import numpy as np

weights_vector = [0.4, 0.6]
first_sample = [282, 226, 188, 327, 344, 304, 414, 224, 335, 270]
second_sample = [417, 851, 742, 1217, 1160, 993, 864, 852, 1286, 988]

stratified_sample = [first_sample, second_sample]

def evaluate_mean(weights: list, stratified_sample: list) -> float:
    xk_exp = [np.mean(sample) for sample in stratified_sample]
    sum_parts = [w_k * exp_x_k for w_k, exp_x_k in zip(weights, xk_exp)]
    
    return sum(sum_parts)

def evaluate_variance(weights: list, stratified_sample: list) -> float:
    weighted_variance = [W_k * np.std(X_k, ddof=0) ** 2 for W_k, X_k in zip(weights, stratified_sample)]
    stratified_mean = evaluate_mean(weights=weights_vector, stratified_sample=stratified_sample)
    weighted_mean_diff = [W_k * (stratified_mean - np.mean(X_k)) ** 2 for W_k, X_k in zip(weights, stratified_sample)]
    
    return sum(weighted_variance) + sum(weighted_mean_diff)

print('Оценка среднего числа свободных мест: ', evaluate_mean(weights=weights_vector, stratified_sample=stratified_sample))
print('Оценка дисперсии числа свободных мест: ', evaluate_variance(weights=weights_vector, stratified_sample=stratified_sample))

result = [282, 226, 188, 327, 344, 304, 414, 224, 335, 270, 417, 851, 742, 1217, 1160, 993, 864, 852, 1286, 988]
print('Выборочное среднее выборки (без стратификации): ', np.mean(result))
print('Выборочная дисперсия (без стратификации): ',np.var(result, ddof=0))

Оценка среднего числа свободных мест:  678.76
Оценка дисперсии числа свободных мест:  136721.6624
Выборочное среднее выборки (без стратификации):  614.2
Выборочная дисперсия (без стратификации):  135461.06


### Объем страт

Генеральная совокупность представляет собой смесь двух распределений: 
<li> первое составляет 25% исходной генеральной совокупности, имеет дисперсию 4, второе - 75%,  дисперсию 6.25. 
<br>
<p>Какими следует выбрать объемы выборок из данных страт так, чтобы оценка мат.ожидания исходной генеральной совокупности имела наименьшую дисперсию, если общий объем выборки должен равняться 100?

<h4>Оптимальное сэмплирование Нейэмана</h4>
<p>$\large n_{k}=n\dfrac{W_{k}\sigma _{k}}{\sum ^{L}_{k=1} {W_k\sigma _{k}}}$

In [2]:
def return_opt_length(weights: list, std_list: list, size: int) -> int:
    parts = [W_k * sigma_k for W_k, sigma_k in zip(weights, std_list)]
    n_list = [round(size * k_sample / sum(parts)) for k_sample in parts]
    
    return n_list

print('Оптимальные объемы разбиения: ', return_opt_length(weights=[0.25, 0.75], std_list=[2, 2.5], size=100))

Оптимальные объемы разбиения:  [21, 79]


In [3]:
10.3 - ((1.21) ** (1/2))/4 * 2.1314, 10.3 + ((1.21) ** (1/2))/4 * 2.1314

(9.713865, 10.886135000000001)

### Доверительные интервалы

Постройте 95%-доверительный интервал для дисперсии $\sigma^2$ ошибок измерений, если  предполагается, что ошибки измерений подчиняются нормальному закону с мат.ожиданием $a=0$ и неизвестной дисперсией $\sigma^2$ 

In [4]:
def return_interval(input_data: list, mu:float, gamma: float) -> tuple:
    from scipy.stats.distributions import chi2
    
    alpha = 1 - gamma
    numerator = sum([(x_i - mu) ** 2 for x_i in input_data])
    return (round(numerator / chi2.ppf(1 - alpha / 2, df=len(input_data)), 3),
            round(numerator / chi2.ppf(alpha / 2, df=len(input_data)), 3))

input_data = [0.47, 0.13, -0.98, 0.74, -2.11, -3.36, -0.35, -2.21, 1.14,
              -0.13, 1.12, 1.49, 0.77, 0.79, 1.13, -1.45, 0.92, -0.08,
              0.62, -0.51]

print('95% доверительный интервал для дисперсии при известном среднем: ',
      return_interval(input_data=input_data, mu=0, gamma=0.95))

95% доверительный интервал для дисперсии при известном среднем:  (0.977, 3.48)


<h4>Вторая задача</h4>
<p>Большая партия изделий может содержать некоторую долю дефектных. Поставщик утверждает, что эта доля составляет 5%, покупатель  предполагает, что доля дефектных изделий равна 10%.

По предложенной выборке постройте 95%-й приближенный доверительный интервал  для доли брака в партии  и ответьте на вопрос:
<li> с большей степенью уверенности можно утверждать, что доля брака  в партии больше 5%?

In [5]:
def return_asympt_interval(input_data: list, gamma: float) -> tuple:
    from scipy.stats.distributions import norm
    
    n = len(input_data)
    p_star = sum(input_data) / n
    alpha = (1 - gamma)
    variance = (p_star * (1 - p_star)) ** (1 / 2)
    return (
        round(p_star - norm.ppf(1 - alpha / 2) * variance / (n ** (1 / 2)), 4),
        round(p_star + norm.ppf(1 - alpha / 2) * variance / (n ** (1 / 2)), 4)
    )

input_data = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

print('Асимптотический 95% доверительный интервал для доли брака в партии: ',
      return_asympt_interval(input_data=input_data, gamma=0.95))

Асимптотический 95% доверительный интервал для доли брака в партии:  (0.0073, 0.0927)
